# Create small netcdf

## Import necessary libraries

In [ ]:
# to open original netcdfs and new netcdf with idx url
import xarray as xr

# backend connecting xarray and openvisuspy
# made one small edit line 203
from backend_v3 import *

## Gather metadata

Get path to a netcdf file containing metadata

In [ ]:
# ******* THIS IS WHEN RUNNING FROM ATLANTIS.SCI **************
# directory to all firesmoke netcdf files
firesmoke_file = "/usr/sci/cedmav/data/firesmoke/BSC00CA12-01/dispersion_20240103.nc"

# # ******* THIS IS WHEN RUNNING FROM MY MACBOOK **************
# # path to firesmoke netcdf file for BSC00CA12-01 dataset, mounted from sci change accordingly
# firesmoke_file = '/Users/arleth/Mount/firesmoke/BSC00CA12-01/dispersion_20240103.nc'

# # or just load locally
# firesmoke_file = "dispersion_20240103.nc"

## Keep all included metadata, EXCEPT replace tsteps, since we have more than 51 AND DROP PM25 as this is stored by idx

### Load the original metadata from one dispersion.nc file

In [ ]:
# load the original netcdf file
ds = xr.open_dataset(firesmoke_file)

# take a look
ds

### Drop the TFLAG and create a new TFLAG variable with all 24k timesteps.* Drop PM25 variable, this is the variable of interest and already in idx file

*timesteps npy file created manually in `firesmoke_tflags.ipynb`

In [ ]:
# drop the TFLAG variable and add our own, also drop PM25, this is the array stored by idx
# ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.drop_vars.html
ds_new = ds.drop_vars("TFLAG")
ds_new = ds_new.drop_vars("PM25")

# ds_new is now empty except for attributes
ds_new

In [ ]:
# import new tflags from file
tflags = np.load("new-firesmoke-tflags.npy")

# reshape them so it's exactly like UBC's original tflags
tflags = np.expand_dims(tflags, axis=1)

# make them int32, as in original netcdf
tflags = tflags.astype(np.int32)

# replace 'TSTEP' with time, to be compatible with our openvisus backend_v3.py
dims = [*ds['TFLAG'].dims]
dims[0] = 'time'

# create a new DataArray using imported tflags, use same attributes and dims from original netcdf file
# ref: https://docs.xarray.dev/en/latest/generated/xarray.Variable.html
# ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html
tflags_var = xr.DataArray(tflags, dims=dims, attrs=ds['TFLAG'].attrs)

# add tflags to dataset
ds_new['TFLAG'] = tflags_var

# take a look, now we just need to add pm25
ds_new

In [ ]:
# Create an PM25 dataarray representing dims of one time slice
pm25_var = xr.DataArray(np.zeros((ds.NROWS, ds.NCOLS)), dims=['ROW', 'COL'], attrs=ds['PM25'].attrs)

# add PM25 to dataset, this is strictly for storing the dimensionality of PM25
# as this is dimensionality info is not available to xarray from idx file
ds_new['PM25'] = pm25_var

### Add idx_url as an attribute, then we're ready to save our tiny netcdf.

In [ ]:
# url to idx file
idx_url = "http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1"

# Adding a new idx_url attribute
ds_new = ds_new.assign_attrs({'idx_url': idx_url})

In [ ]:
# directory to new tiny netcfd file
new_nc = "firesmoke_metadata.nc"

# save xarray dataset as netcdf, this is the metadata we put into tiny netcdf
ds_new.to_netcdf("firesmoke_metadata.nc")

In [ ]:
# check out the tiny netcdf loaded by vanilla xarray
xr.open_dataset(new_nc)

## Test tiny netcdf

## Load tiny netcdf file with backend_v3 and pass timestamps above as timesteps

In [ ]:
# load data
final_ds = xr.open_dataset(new_nc, engine=OpenVisusBackendEntrypoint)
final_ds